<a href="https://colab.research.google.com/github/Dohy-Lee/ML_DL.Basic/blob/main/5_3)Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#정형 데이터(Structured Data) : 어떤 구조로 되어 있음. CSV, DB, 엑셀에 저장하기 쉬움
#비정형 데이터(Unstructured Data) : DB나 엑셀로 표현하기 어려운 것들.
#앙상블 학습(Ensemble Learning) : 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘. 대부분 결정 트리 기반으로 만들어져있음.
#랜덤 포레스트(Random Forest) : 앙상블 학습의 대표적인 모델. 무작위로 택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고, 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음.
#부트스트랩 샘플(Bootstrap Sample) : 훈련 데이터에서 무작위로 샘플을 추출하는데, 이 때 한 샘플을 중복하여 뽑을 수 있음.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size = 0.2, random_state = 42)

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [7]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [8]:
#OOB(Out Of Bag) : 부트스트랩 샘플에 포함되지 않고 남는 샘플. 이 샘플로 검증 세트마냥 부트스트랩 샘플로 훈련한 모델을 평가 할 수 있음
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [9]:
#엑스트라 트리(Extra Trees) : 랜덤 포레스트와 유사하지만, 부트스트랩을 사용하지않는다는 큰 차이가 있음.(즉 전체 훈련 세트를 사용하여 각 결정 트리를 만듦) 또한, 노드를 분할할 때 무작위로 분할.
#특성을 무작위로 분할 : 성능은 낮아지지만, 많은 트리를 앙상블 하기때문에 과대적합을 막고, 검증 세트의 점수를 높이는 효과가 있음. 또한, 무작위로 노드를 분할하기에 계산속도가 빠름

In [10]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1 , random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs  = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [12]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [13]:
#그레이디언트 부스팅(Gradient Boosting) : 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법. 깊이가 얕기때문에 과대적합에 강하고, 일반적으로 높은 일반화 성능을 기대할 수 있음.
#                                         분류에서는 로지스틱 손실 함수, 회귀에서는 평균 제곱 오차 함수 사용 

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [20]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [18]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [22]:
#히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting) : 그레이디언트 부스팅의 속도와 성능을 개선함.
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [23]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [26]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs = -1)
print(result.importances_mean)
hgb.score(test_input, test_target)

[0.05969231 0.20238462 0.049     ]


0.8723076923076923

In [28]:
#사이킷런이외 그레이디언트 라이브러리 1. XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [30]:
#사이킷런이외 그레이디언트 라이브러리 2. LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
